In [2]:
import pandas as pd
import numpy as np

In [3]:
sales = pd.read_excel("./data.xlsx")  # data.xlsx
fcst  = pd.read_excel("./Forcast.xlsx")  # Forcast.xlsx

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678025 entries, 0 to 678024
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Site              678021 non-null  object 
 1   Date              678019 non-null  object 
 2   Invoice Number    678013 non-null  object 
 3   Customer Code     678002 non-null  object 
 4   Name              677996 non-null  object 
 5   Item Code         677993 non-null  object 
 6   Item Description  677983 non-null  object 
 7   Quantity          677916 non-null  float64
dtypes: float64(1), object(7)
memory usage: 41.4+ MB


In [5]:
fcst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BRAND                24 non-null     object 
 1   ACTIVE               24 non-null     object 
 2   Product              24 non-null     object 
 3   Product description  24 non-null     object 
 4   PROD                 24 non-null     object 
 5   TYP                  24 non-null     object 
 6   STATUS               24 non-null     object 
 7   ON HAND INITIAL      0 non-null      float64
 8   PO INITIAL           0 non-null      float64
 9   OCT                  18 non-null     float64
 10  NOV                  18 non-null     float64
 11  DEC                  24 non-null     int64  
 12  JAN                  24 non-null     int64  
 13  FEB                  23 non-null     float64
 14  MAR                  23 non-null     float64
 15  APR                  24 non-null     int64

In [6]:
# Keep only needed columns (robust to case/whitespace)
def pick(df, names):
    cols = {c.strip().lower(): c for c in df.columns}
    out = {}
    for n in names:
        key = n.strip().lower()
        if key in cols: out[n] = cols[key]
    return df[[out[n] for n in out]]

sales = pick(sales, ["Site","Date","Invoice Number","Customer Code","Name","Item Code","Item Description","Quantity"])
fcst  = pick(fcst , ["BRAND","ACTIVE","Product","Product description","PROD","TYP","STATUS"]) \
        .join(pd.read_excel("./Forcast.xlsx").drop(columns=["BRAND","ACTIVE","Product","Product description","PROD","TYP","STATUS"], errors="ignore"))

In [8]:
fcst.columns

Index(['BRAND', 'ACTIVE', 'Product', 'Product description', 'PROD', 'TYP',
       'STATUS', 'ON HAND INITIAL', 'PO INITIAL', 'OCT', 'NOV', 'DEC', 'JAN',
       'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT.1',
       'NOV.1', 'DEC.1', 'JAN.1', 'FEB.1', 'MAR.1', 'APR.1', 'MAY.1', 'JUN.1',
       'JULY', 'AUG.1', 'SEPT.1', 'OCT.2', 'NOV.2', 'DEC.2', 'JAN.2', 'FEB.2',
       'MAR.2', 'APR.2', 'MAY.2', 'JUN.2', 'JUL.1', 'AUG.2', 'SEPT.2', 'OCT.3',
       'NOV.3', 'DEC.3', 'JAN.3', 'FEB.3', 'MAR.3', 'APR.3', 'MAY.3', 'JUN.3',
       'JUL.2', 'AUG.3', 'SEPT.3', 'OCT.4', 'NOV.4', 'DEC.4', 'JAN.4', 'FEB.4',
       'MAR.4', 'APR.4', 'MAY.4', 'JUN.4', 'JUL.3', 'AUG.4', 'SEP', 'OCT.5',
       'NOV.5', 'DEC.5'],
      dtype='object')

In [24]:
# EDA + Data Sanity + Linkage - 
# Sets of IDs to compare (case-insensitive)
sales["item_code_std"] = sales["Item Code"].astype(str).str.upper().str.strip()
fcst["product_std"]    = fcst["Product"].astype(str).str.upper().str.strip()

# --- Find exact matches ---
direct_codes = sorted(set(sales["item_code_std"]) & set(fcst["product_std"]))
print("Direct matches:", len(direct_codes))
print(len(sales["item_code_std"]))
print(len(fcst["product_std"]))

Direct matches: 8
678025
24


In [ ]:
# All forcast products - 8 are present in sales, but not all, hence forcast for missing products in Sales will 
# be missing.

In [18]:
neg_rows = sales[sales["Quantity"]<0]
print(len(sales), len(neg_rows))


678025 12611


In [22]:
# Building linkage table - 
link = pd.DataFrame({"item_code_std": list(direct)})
link = link.merge(fcst[["product_std"]].drop_duplicates(),
                  left_on="item_code_std", right_on="product_std", how="left")
link = link.rename(columns={"product_std":"product_link"})
assert not link["item_code_std"].duplicated().any(), "Duplicate key in linkage table!"


KeyError: "None of [Index(['product_std'], dtype='object')] are in the [columns]"